In [711]:
import numpy as np
import pandas as pd
from math import radians
import sklearn.metrics

In [712]:
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')

print(f"Train size: {df_train.shape[0]}")
print(f"Test size: {df_test.shape[0]}")

Train size: 20254
Test size: 6966


Primary School

In [713]:
df_pri_sch = pd.read_csv('../data/auxiliary-data/sg-primary-schools.csv')
df_pri_sch = df_pri_sch.drop(["name", "lat", "lng", 'planning_area'], axis=1)
pri_sch_cleaned = df_pri_sch.value_counts().to_frame(name="pri_sch")

#TODO: Get schools in vicinity instead of just subzone.

Secondary School

In [714]:
df_sec_sch = pd.read_csv('../data/auxiliary-data/sg-secondary-schools.csv')
df_sec_sch = df_sec_sch.drop(["name", "lat", "lng", 'planning_area'], axis=1)
sec_sch_cleaned = df_sec_sch.value_counts().to_frame(name="sec_sch")

#TODO: Get schools in vicinity instead of just subzone.

Population Density of Subzone

In [715]:
df_subzone = pd.read_csv('../data/auxiliary-data/sg-subzones.csv')
df_subzone["population_density"] = df_subzone['population']/df_subzone["area_size"]
df_subzone = df_subzone.drop(['area_size', 'population', 'planning_area'],axis=1).set_index("name")

In [716]:
df_dirty_test = df_test
df_dirty_train = df_train

Remove duplicates and invalid data (Rows)


In [717]:
df_cleaned = df_dirty_train.drop_duplicates()
df_cleaned = df_cleaned[df_cleaned.size_sqft > 0]
df_cleaned.dropna(subset=['num_beds', 'num_baths', 'price', 'size_sqft', 'built_year', 'available_unit_types', 'tenure'], inplace = True)
df_cleaned = df_cleaned[df_cleaned.price > 0]
df_cleaned = df_cleaned[df_cleaned.furnishing != "na"]
print(f'Records dropped :{df_dirty_train.shape[0] - df_cleaned.shape[0]}' )


Records dropped :3034


#c.	Standardize capitalization


In [718]:
df_cleaned['property_type'] = df_cleaned['property_type'].str.lower()
df_cleaned['tenure'] = df_cleaned['tenure'].str.lower()
df_cleaned['furnishing'] = df_cleaned['furnishing'].str.lower()
df_cleaned['subzone'] = df_cleaned['subzone'].str.lower()
df_cleaned['planning_area'] = df_cleaned['planning_area'].str.lower()

df_cleaned['built_year'] = df_cleaned['built_year'].astype(int)
df_cleaned['num_beds'] = df_cleaned['num_beds'].astype(int)
df_cleaned['num_baths'] = df_cleaned['num_baths'].astype(int)
df_cleaned['lng'] = df_cleaned['lng'].astype(np.float16)
df_cleaned['lat'] = df_cleaned['lat'].astype(np.float16)

df_test_cleaned = df_dirty_test
df_test_cleaned['property_type'] = df_test_cleaned['property_type'].str.lower()
df_test_cleaned['tenure'] = df_test_cleaned['tenure'].str.lower()
df_test_cleaned['furnishing'] = df_test_cleaned['furnishing'].str.lower()
df_test_cleaned['subzone'] = df_test_cleaned['subzone'].str.lower()
df_test_cleaned['planning_area'] = df_test_cleaned['planning_area'].str.lower()

#TODO: Fix comments
#df_test_cleaned['built_year'] = df_test_cleaned['built_year'].astype(int)
#df_test_cleaned['num_beds'] = df_test_cleaned['num_beds'].astype(int)
#df_test_cleaned['num_baths'] = df_test_cleaned['num_baths'].astype(int)
df_test_cleaned['lng'] = df_test_cleaned['lng'].astype(np.float16)
df_test_cleaned['lat'] = df_test_cleaned['lat'].astype(np.float16)


Clean Lease tenure column, Property_Type Column

In [719]:
#TENURE COLUMN
mask_999 = ['947-year leasehold', '929-year leasehold', '946-year leasehold',
'956-year leasehold']
mask_99 = ['100-year leasehold', '102-year leasehold', '110-year leasehold', '103-year leasehold']
df_cleaned = df_cleaned.replace(mask_999, '999-year leasehold')
df_cleaned = df_cleaned.replace(mask_99, '99-year leasehold')

df_test_cleaned = df_test_cleaned.replace(mask_999, '999-year leasehold')
df_test_cleaned = df_test_cleaned.replace(mask_99, '99-year leasehold')

print(f"Train size: {df_cleaned.shape[0]}")
print(f"Test size: {df_test_cleaned.shape[0]}")


Train size: 17220
Test size: 6966


In [720]:
#PROPERTY TYPE
# changing "hdb 3 rooms", "hdb 4 rooms" and likewise to "hdb" since the number of rooms info can
# be obtained from "num_beds"
df_cleaned['property_type'].mask(df_cleaned['property_type'].str.contains("hdb"), "hdb", inplace=True)
df_cleaned.drop(df_cleaned[df_cleaned['property_type']  == 'land only'].index, inplace = True)

df_test_cleaned['property_type'].mask(df_test_cleaned['property_type'].str.contains("hdb"), "hdb", inplace=True)

#TODO: Reduce number of property types with less properties. Maybe do an EDA and figure out best way to remove them. Perhaps club 
#them in different category.

df_cleaned.loc[(df_cleaned['property_type']=="apartment") & 
       (df_cleaned['title'].str.contains('condo')),['property_type']] = "condo"

df_cleaned.loc[(df_cleaned['property_type']=="walk-up") & 
       (df_cleaned['title'].str.contains('condo')),['property_type']] = "condo"

df_cleaned.loc[(df_cleaned['property_type']=="good class bungalow"),['property_type']] = "bungalow"

#Test
df_test_cleaned.loc[(df_test_cleaned['property_type']=="apartment") & 
       (df_test_cleaned['title'].str.contains('condo')),['property_type']] = "condo"

df_test_cleaned.loc[(df_test_cleaned['property_type']=="walk-up") & 
       (df_test_cleaned['title'].str.contains('condo')),['property_type']] = "condo"

df_test_cleaned.loc[(df_test_cleaned['property_type']=="good class bungalow"),['property_type']] = "bungalow"

df_test_cleaned.loc[(df_test_cleaned['property_type']=="conservation house") & 
       (df_test_cleaned['property_details_url'].str.contains('condo')),['property_type']] = "condo"

df_test_cleaned.loc[(df_test_cleaned['property_type']=="good class bungalow"),['property_type']] = "bungalow"

#The following results have been picked manually from the website 99.co
df_test_cleaned.loc[(df_test_cleaned['property_type']=="conservation house") & 
       (df_test_cleaned['property_details_url'].str.contains('blair-plain-conservation-area')),['property_type']] = "landed"

df_test_cleaned.loc[(df_test_cleaned['property_type']=="conservation house") & 
       (df_test_cleaned['property_details_url'].str.contains('beng-tong-mansion')),['property_type']] = "landed"

print(f"Train size: {df_cleaned.shape[0]}")
print(f"Test size: {df_test_cleaned.shape[0]}")


Train size: 17219
Test size: 6966


Handle Missing Values

In [721]:
#h. Handle missing values

#built_year
dfmap = df_cleaned.dropna(subset = ['built_year'])[['built_year', 'property_name']].drop_duplicates()
dfmap = dfmap.drop_duplicates(subset = ['property_name'])
df_cleaned = df_cleaned.drop(columns=['built_year']).merge(dfmap, on=['property_name'], how='left')

dfmap = df_cleaned.dropna(subset = ['tenure'])[['tenure', 'property_name']].drop_duplicates()
dfmap = dfmap.drop_duplicates(subset = ['property_name'])
df_cleaned = df_cleaned.drop(columns=['tenure']).merge(dfmap, on=['property_name'], how='left')

dfmap = df_cleaned.dropna(subset = ['tenure'])[['tenure', 'address']].drop_duplicates()
dfmap = dfmap.drop_duplicates(subset = ['address'])
df_cleaned = df_cleaned.drop(columns=['tenure']).merge(dfmap, on=['address'], how='left')

#df_cleaned.isnull().sum()



In [722]:
#h. Handle missing values
#print(df_test_cleaned.isnull().sum())
print("----------------------------------------------------")

#built_year
dfmap = df_test_cleaned.dropna(subset = ['built_year'])[['built_year', 'property_name']].drop_duplicates()
dfmap = dfmap.drop_duplicates(subset = ['property_name'])
df_test_cleaned = df_test_cleaned.drop(columns=['built_year']).merge(dfmap, on=['property_name'], how='left')


#df_test_cleaned = df_test_cleaned.dropna(subset=['built_year'])

dfmap = df_test_cleaned.dropna(subset = ['tenure'])[['tenure', 'property_name']].drop_duplicates()
dfmap = dfmap.drop_duplicates(subset = ['property_name'])
df_test_cleaned = df_test_cleaned.drop(columns=['tenure']).merge(dfmap, on=['property_name'], how='left')

dfmap = df_test_cleaned.dropna(subset = ['tenure'])[['tenure', 'address']].drop_duplicates()
dfmap = dfmap.drop_duplicates(subset = ['address'])
df_test_cleaned = df_test_cleaned.drop(columns=['tenure']).merge(dfmap, on=['address'], how='left')

#Fill in number of baths from number of bedrooms.
df_test_cleaned.loc[(df_test_cleaned.num_baths.isnull() & df_test_cleaned.num_beds == 1), ['num_baths']] = 1
df_test_cleaned.loc[(df_test_cleaned.num_baths.isnull() & df_test_cleaned.num_beds == 2), ['num_baths']] = 2
df_test_cleaned.loc[(df_test_cleaned.num_baths.isnull() & df_test_cleaned.num_beds > 2), ['num_baths']] = df_test_cleaned.num_beds -1

#Fill in number of beds from number of bathrooms.
df_test_cleaned.loc[(df_test_cleaned.num_beds.isnull() & df_test_cleaned.num_baths == 1), ['num_beds']] = 1
df_test_cleaned.loc[(df_test_cleaned.num_beds.isnull() & df_test_cleaned.num_baths == 2), ['num_beds']] = 2
df_test_cleaned.loc[(df_test_cleaned.num_beds.isnull() & df_test_cleaned.num_baths > 2), ['num_beds']] = df_test_cleaned.num_baths + 1

#There are 5 properties which have both Nan values for bed and baths. Estimate their beds from area.
df_test_cleaned.loc[(df_test_cleaned.num_beds.isnull() & df_test_cleaned.size_sqft < 600), ['num_beds']] = 1
df_test_cleaned.loc[(df_test_cleaned.num_beds.isnull() & df_test_cleaned.size_sqft < 1000), ['num_beds']] = 2
df_test_cleaned.loc[(df_test_cleaned.num_beds.isnull() & df_test_cleaned.size_sqft < 1700), ['num_beds']] = 3
df_test_cleaned.loc[(df_test_cleaned.num_beds.isnull() & df_test_cleaned.size_sqft < 3000), ['num_beds']] = 4

#Estimate their baths from beds.
df_test_cleaned.loc[(df_test_cleaned.num_baths.isnull() & df_test_cleaned.num_beds == 1), ['num_baths']] = 1
df_test_cleaned.loc[(df_test_cleaned.num_baths.isnull() & df_test_cleaned.num_beds == 2), ['num_baths']] = 2
df_test_cleaned.loc[(df_test_cleaned.num_baths.isnull() & df_test_cleaned.num_beds > 2), ['num_baths']] = df_test_cleaned.num_beds -1


----------------------------------------------------


Singapore has the following latitude and longitude coordinates in its extreme ends:
1. left-most (Tuas) :  1.30871,103.64287
2. right-most (Changi) : 1.34538,104.00270
3. top-most (Sembawang) : 1.46227,103.79487
4. bottom-most (Bukit Merah) : 1.28762,103.82467


#### Min latitude - 1.28762       Max latitude - 1.46227

#### Min longitude - 103.64         Max longitude - 104.00

But we can see that in the data, min longitude is -77.065364 and max latitude is 69.486768 which are out of the range of latitude and longitude values 

<img src="images/singapore-lat-long-map.jpeg" width=600 height=600 />

In [723]:
df_max_lng = df_cleaned[df_cleaned.lng > 121.0]
df_min_lng = df_cleaned[df_cleaned.lng < -77.0]
df_max_lat = df_cleaned[df_cleaned.lat > 69.0]
df_wrong_coordinates = pd.concat([df_max_lng, df_min_lng, df_max_lat])

print("It is interesting to note that in all the records where latitude and longitude have incorrect coordinates, planning_area and subzone have missing values, this can also be verified by checking for count of missing values")
print(df_wrong_coordinates["address"].value_counts())
# coordinates are incorrect for 5 'address'


# using the 'address' we can manually correct the latitude, longitude coordinates along with 
# filling of values for sub zone and planning_area

df_cleaned.loc[df_cleaned.address == "1 tessensohn road", 
               ['property_type', 'lat', 'lng', 'subzone', 'planning_area']] = 'condo', 1.3164313, 103.8575321, 'balestier', 'novena'
df_cleaned.loc[df_cleaned.address == "38 lorong 32 geylang", 
               ['property_type', 'lat', 'lng', 'subzone', 'planning_area']] = 'condo', 1.31262, 103.88686, 'aljunied', 'geylang'
df_cleaned.loc[df_cleaned.address == "5 jalan mutiara", 
               ['property_type', 'lat', 'lng', 'subzone', 'planning_area']] = 'condo', 1.29565, 103.82887, 'leonie hill', 'river valley'
df_cleaned.loc[df_cleaned.address == "17 farrer drive", 
               ['property_type', 'lat', 'lng', 'subzone', 'planning_area']] = 'condo', 1.313259, 103.806622, 'holland road', 'bukit timah'
df_cleaned.loc[df_cleaned.address == "15 farrer drive", 
               ['property_type', 'lat', 'lng', 'subzone', 'planning_area']] = 'condo', 1.313259, 103.806622, 'holland road', 'bukit timah'

It is interesting to note that in all the records where latitude and longitude have incorrect coordinates, planning_area and subzone have missing values, this can also be verified by checking for count of missing values
38 lorong 32 geylang    6
17 farrer drive         3
15 farrer drive         2
Name: address, dtype: int64


In [724]:
df_max_lng = df_test_cleaned[df_test_cleaned.lng > 121.0]
df_min_lng = df_test_cleaned[df_test_cleaned.lng < -77.0]
df_max_lat = df_test_cleaned[df_test_cleaned.lat > 69.0]
df_wrong_coordinates = pd.concat([df_max_lng, df_min_lng, df_max_lat])

print("It is interesting to note that in all the records where latitude and longitude have incorrect coordinates, planning_area and subzone have missing values, this can also be verified by checking for count of missing values")
print(df_wrong_coordinates["address"].value_counts())
# coordinates are incorrect for 5 'address'


# using the 'address' we can manually correct the latitude, longitude coordinates along with 
# filling of values for sub zone and planning_area

df_test_cleaned.loc[df_test_cleaned.address == "38 lorong 32 geylang", 
               ['property_type', 'lat', 'lng', 'subzone', 'planning_area']] = 'condo', 1.31262, 103.88686, 'aljunied', 'geylang'
df_test_cleaned.loc[df_test_cleaned.address == "17 farrer drive", 
               ['property_type', 'lat', 'lng', 'subzone', 'planning_area']] = 'condo', 1.313259, 103.806622, 'holland road', 'bukit timah'
        

It is interesting to note that in all the records where latitude and longitude have incorrect coordinates, planning_area and subzone have missing values, this can also be verified by checking for count of missing values
38 lorong 32 geylang    2
17 farrer drive         1
Name: address, dtype: int64


Merging With Auxiliary Data

In [725]:
df_mrt_station = pd.read_csv('../data/auxiliary-data/sg-mrt-stations.csv')
df_mrt_station = df_mrt_station.drop(["name", "lat", "lng", 'planning_area', 'code', 'line', 'opening_year'], axis=1)
mrt_station_cleaned = df_mrt_station.value_counts().to_frame(name="mrt_station")

mrt_station_coor = pd.read_csv('../data/auxiliary-data/sg-mrt-stations.csv').drop(["name", "subzone", 'planning_area', 'code', 'line', 'opening_year'], axis=1).to_numpy()
mrt_station_coor = np.array([[radians(_) for _ in coor] for coor in mrt_station_coor])

df_train_coor = df_cleaned[["lat", "lng"]].to_numpy()
df_train_coor = np.array([[radians(_) for _ in coor] for coor in df_train_coor])

df_test_coor = df_test_cleaned[["lat", "lng"]].to_numpy()
df_test_coor = np.array([[radians(_) for _ in coor] for coor in df_test_coor])

dist_matrix_train = sklearn.metrics.pairwise.haversine_distances(mrt_station_coor, df_train_coor)
# multiply to get meters
closest_dist_to_mrt_train = pd.DataFrame(np.amin(dist_matrix_train, axis=0)* 6371000, columns=["closest_dist_to_mrt"])

dist_matrix_test = sklearn.metrics.pairwise.haversine_distances(mrt_station_coor, df_test_coor)
# multiply to get meters
closest_dist_to_mrt_test = pd.DataFrame(np.amin(dist_matrix_test, axis=0)* 6371000, columns=["closest_dist_to_mrt"])

df_cleaned = df_cleaned.merge(pri_sch_cleaned, how='left',left_on="subzone",right_on="subzone")\
    .merge(sec_sch_cleaned, how='left',left_on="subzone",right_on="subzone")\
    .merge(mrt_station_cleaned, how='left',left_on="subzone",right_on="subzone")\
    .merge(df_subzone, how='left',left_on="subzone",right_on="name")\
    .join(closest_dist_to_mrt_train)\
    .fillna({'pri_sch':0, 'sec_sch':0, 'mrt_station':0})

df_test_cleaned = df_test_cleaned.merge(pri_sch_cleaned, how='left',left_on="subzone",right_on="subzone")\
    .merge(sec_sch_cleaned, how='left',left_on="subzone",right_on="subzone")\
    .merge(mrt_station_cleaned, how='left',left_on="subzone",right_on="subzone")\
    .merge(df_subzone, how='left',left_on="subzone",right_on="name")\
    .join(closest_dist_to_mrt_test)\
    .fillna({'pri_sch':0, 'sec_sch':0, 'mrt_station':0})

Drop Columns at the end

In [726]:
df_cleaned = df_cleaned.drop(['title', 'address','property_name', 'property_details_url', 'listing_id', 'elevation', 'total_num_units', 'floor_level', 'available_unit_types'], axis = 1)
df_test_cleaned = df_test_cleaned.drop(['title','address','property_name', 'property_details_url', 'listing_id', 'elevation', 'total_num_units', 'floor_level', 'available_unit_types'], axis = 1)

In [727]:
print(df_cleaned.isnull().sum())
print(df_test_cleaned.isnull().sum())
print(df_cleaned.shape)
print(df_test_cleaned.shape)

property_type          0
num_beds               0
num_baths              0
size_sqft              0
furnishing             0
lat                    0
lng                    0
subzone                0
planning_area          0
price                  0
built_year             0
tenure                 0
pri_sch                0
sec_sch                0
mrt_station            0
population_density     0
closest_dist_to_mrt    0
dtype: int64
property_type            0
num_beds                 0
num_baths                0
size_sqft                0
furnishing               0
lat                      0
lng                      0
subzone                 30
planning_area           30
built_year             341
tenure                  78
pri_sch                  0
sec_sch                  0
mrt_station              0
population_density      30
closest_dist_to_mrt      0
dtype: int64
(17219, 17)
(6966, 16)


Saving the data

In [728]:
df_test_cleaned.to_csv('../data/test_cleaned.csv', index = False)
df_cleaned.to_csv('../data/train_cleaned.csv', index = False)

In [729]:
df_cleaned.isnull().sum()

property_type          0
num_beds               0
num_baths              0
size_sqft              0
furnishing             0
lat                    0
lng                    0
subzone                0
planning_area          0
price                  0
built_year             0
tenure                 0
pri_sch                0
sec_sch                0
mrt_station            0
population_density     0
closest_dist_to_mrt    0
dtype: int64

In [731]:
df_test_cleaned.isnull().sum()
df_test_cleaned.shape


(6966, 16)